# Robust Identification of Investor Beliefs

by [Xiaohong Chen](https://economics.yale.edu/people/faculty/xiaohong-chen), [Lars Peter Hansen](http://larspeterhansen.org/) and [Peter G. Hansen](https://mitsloan.mit.edu/phd/students/peter-hansen).

The latest draft of the paper can be found [here](http://larspeterhansen.org/research/papers/).

Notebook by: Han Xu

## 1. Overview

This notebook provides source code and explanations for how we solve the dynamic problem in Section 5. It also provides source code for the tables in Section 6. 

## 2. Intertemporal Divergence

Recall Problem 5.4 in the paper:

\begin{equation}
\mu = \min_{N_1\in \mathcal{N}} \mathbb{E}\left(N_1\left[g(X_1)+\xi\log N_1 + v_1\right]\mid \mathfrak{I}_0\right) - v_0
\end{equation}
subject to constraints:
\begin{equation}
\mathbb{E}\left[N_1 f(X_1)\mid\mathfrak{I}_0\right] = 0
\end{equation}

By Proposition 5.6, this problem can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_{\lambda_0}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{I}_0\right)
\end{equation}

where
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

The implied solution for the probablity distortion is:

\begin{equation}
N_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\lambda^*_0(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

where $\lambda^*_0$ is the optimizing choice for $\lambda_0$ and $\left(\epsilon^*,e_0^*\right)$ are selected to that the resulting $\sf Q$ implies that the process is stochastically stable. The conditional expectation implied by the bound is

\begin{equation}
\mathbb{E}\left[N_1^*g(X_1)\mid \mathfrak{I}_0\right]
\end{equation}

which in turn implies a bound on the unconditional expectation equal to

\begin{equation}
\int \mathbb{E}\left[N_1^*g(X_1)\mid\mathfrak{I}_0\right]d \sf Q_0^*
\end{equation}

The implied relative entropy is

\begin{equation}
\int \mathbb{E}\left(N_1^*\log N_1^*\mid \mathfrak{I}_0\right)d \sf Q_0^*
\end{equation}

## 3. Code Implementation

In [ ]:
# Load packages
import time
from utilities import *
from plotting_module import *

### 3.1 Data
The file “UnitaryData.csv” contains the following variables:

- The first four columns contain Euler equation errors from the unitary risk aversion model corresponding to the 3-month T-bill rate, the market excess return, the SMB excess return, and the HML excess return respectively. Under a feasible belief distortion, all four of these variables should have expectation zero (conditional or unconditional).


- The column “d.p” contains the dividend-price ratio for the CRSP value weighted index, computed at the start of the return period. Hence functions of d.p[i] (i.e. quantile indicator functions) are valid instruments for the returns in row i.


- The final column “log.RW” contains values of the logarithmic return on wealth. This is the random variable who’s expectation we are intersted in bounding.

All returns are quarterly and inflation adjusted.

In [ ]:
# Load data
data = pd.read_csv('UnitaryData.csv')
# Show statistics of the data
data.describe()

We first compute the terciles of dividend-price ratio and form indicator functions of these terciles. Then we multiply the first four columns by the indicator functions to form a 12-dimensional $f$. The last column log.RW is our $g$.

### 3.2 Computational Strategy

In our application, we just have 3 distinct states, so we can represent the function $e(\cdot)$ as a 3-dimensional vector. Additionally, we are free to impose the normalization $e_1=1$. We can solve the dual problem numerically by something analogous to value function iteration for $e=(1,e_2,e_3)$. Here is the iteration scheme:

1\. Guess $e=(1,1,1)$.

2\. For $k \in \{1,2,3\}$, solve
\begin{equation}
v_k = \min_{\lambda_0} \hat{\mathbb{E}}\left(\exp \left[-\frac{1}{\xi}g(X_1) + \lambda_0f(X_1)\right]e(X_1)\mid \text{state today = } k\right)
\end{equation}

3\. Store
\begin{align*}
\hat{e} &= v/v_1 \\
\hat{\epsilon} &= v_1 \\
\text{error} &= \|\hat{e}-e\|
\end{align*}

4\. Set $e = \hat{e}$.

5\. Iterate steps 2-4 until error is arbitrarily close to zero.

Once we have (approximately) stationary values for $\epsilon$ and $e$, we can form the conditional belief distortion
\begin{equation}
N_1 = \frac{1}{\epsilon} \exp \left[-\frac{1}{\xi}g(X_1)+\lambda_0 \cdot f(X_1)\right]\frac{e_1}{e_0}
\end{equation}

To obtain the unconditional relative entropy, we need to average across states using the implied stationary distribution coming from the distorted probabilities. Define a $3\times 3$ matrix $\tilde{P}$ by 
$$
\tilde{P}_{i,j} = \hat{\mathbb{E}}\left[N_1 \mathcal{1}\left(\text{state tomorrow = j}\right)\mid \text{state today = i}\right]
$$

We should have that $\tilde{P}$ is a transition probability matrix, so $\tilde{P}\mathcal{1}=\mathcal{1}$. Next, solve for the stationary distribution $\pi\in \mathbb{R}^3$ as the dominant left eigenvector of $\tilde{P}$, i.e.
\begin{equation}
\tilde{\pi}^\prime \tilde{P} = \tilde{\pi}^\prime
\end{equation}

Then, the unconditional relative entropy can be computed as
\begin{equation}
\text{RE}(\xi) = \sum_{k=1}^{3}\hat{\mathbb{E}}\left[N_1\log N_1 \mid \text{state today = k}\right]\cdot \tilde{\pi}_k
\end{equation}

In [ ]:
# Initialize the solver
solver = InterDivConstraint(tol=1e-10,max_iter=1000)

# Define g(X) = log Rw
solver.g = solver.log_Rw

# Set ξ, corresponding to 110% min RE
ξ = 0.2  

time_start = time.time() 
result_lower = solver.iterate(ξ,lower=True)
result_upper = solver.iterate(ξ,lower=False)
time_spent = round(time.time()-time_start,4)

### 3.3 Results

In [ ]:
# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (time_spent))

# Print converged parameter results
print("\n")
print("--- Converged vlues for the lower bound problem ---")
print("ϵ: %s" % result_lower['ϵ'])
print("e: %s" % result_lower['e'])
print("λ: %s" % result_lower['λ'])
print("μ: %s" % result_lower['μ'])

print(" ")
print("--- Converged vlues for the upper bound problem ---")
print("ϵ: %s" % result_upper['ϵ'])
print("e: %s" % result_upper['e'])
print("λ: %s" % result_upper['λ'])
print("μ: %s" % result_upper['μ'])

# Print transition probability matrix under the original empirical probability
print("\n")
print("--- Transition Probability Matrix (Original) ---")
print(result_lower['P'])

# Print transition probability matrix under distorted probability, lower bound
print(" ")
print("--- Transition Probability Matrix (Distorted, lower bound problem) ---")
print(result_lower['P_tilde'])

# Print transition probability matrix under distorted probability, upper bound
print(" ")
print("--- Transition Probability Matrix (Distorted, upper bound problem) ---")
print(result_upper['P_tilde'])


# Print stationary distribution under the original empirical probability
print("\n")
print("--- Stationary Distribution (Original) ---")
print(result_lower['π'])

# Print stationary distribution under distorted probability, lower bound
print(" ")
print("--- Stationary Distribution (Distorted, lower bound problem) ---")
print(result_lower['π_tilde'])

# Print stationary distribution under distorted probability, upper bound
print(" ")
print("--- Stationary Distribution (Distorted, upper bound problem) ---")
print(result_upper['π_tilde'])

# Print relative entropy
print("\n")
print("--- Relative Entropy (lower bound problem) ---")
print("E[MlogM|state 1] = %s " % result_lower['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_lower['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_lower['RE_cond'][2])
print("E[MlogM]         = %s " % result_lower['RE'])

# Print relative entropy
print(" ")
print("--- Relative Entropy (Upper bound problem) ---")
print("E[MlogM|state 1] = %s " % result_upper['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result_upper['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result_upper['RE_cond'][2])
print("E[MlogM]         = %s " % result_upper['RE'])

# Print conditional moment & bounds
print("\n")
print("--- Moment (Original, annualized, %) ---")
print("E[g(X)|state 1]  = %s " % (result_lower['moment_cond'][0]*400))
print("E[g(X)|state 2]  = %s " % (result_lower['moment_cond'][1]*400))
print("E[g(X)|state 3]  = %s " % (result_lower['moment_cond'][2]*400))
print("E[g(X)]  = %s " % (result_lower['moment']*400))
print(" ")
print("--- Moment (Lower bound, annualized, %) ---")
print("E[Mg(X)|state 1] = %s " % (result_lower['moment_bound_cond'][0]*400))
print("E[Mg(X)|state 2] = %s " % (result_lower['moment_bound_cond'][1]*400))
print("E[Mg(X)|state 3] = %s " % (result_lower['moment_bound_cond'][2]*400))
print("E[Mg(X)] = %s " % (result_lower['moment_bound']*400))
print(" ")
print("--- Moment (Upper bound, annualized, %) ---")
print("E[Mg(X)|state 1] = %s " % (result_upper['moment_bound_cond'][0]*400))
print("E[Mg(X)|state 2] = %s " % (result_upper['moment_bound_cond'][1]*400))
print("E[Mg(X)|state 3] = %s " % (result_upper['moment_bound_cond'][2]*400))
print("E[Mg(X)] = %s " % (result_upper['moment_bound']*400))

### 3.4 Plots

In [ ]:
time_start = time.time() 
entropy_moment_bounds()
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

conditioning = ['low D/P', 'middle D/P', 'high D/P', 'unconditional']
min_entropy_implied = np.array([2.18, 2.73, 4.80, 2.40])
empirical_average = np.array([5.12, 3.54, 13.90, 7.54])

#each number is repeated once in order to form a desirable shape of boxes
low_bound_20 = np.array([1.54, 1.54, 2.96, 2.96]) 
middle_bound_20 = np.array([2.54, 2.54, 2.93, 2.93])
high_bound_20 = np.array([4.54,4.54,5.10,5.10])
unconditional_bound_20 = np.array([1.72, 1.72, 3.13, 3.13])

fig, ax = plt.subplots()
bplot = ax.boxplot(np.vstack((low_bound_20, middle_bound_20, high_bound_20, unconditional_bound_20)).T,
                   usermedians = min_entropy_implied, #override the automatically calculated median
                   labels = conditioning, widths = 0.4, 
                   medianprops = dict(linestyle='-.', linewidth = 0, color = 'black'),
                   patch_artist = True)

for box in bplot['boxes']:
    box.set(facecolor = 'salmon')

splot = ax.scatter(x = np.arange(1, 5), y = empirical_average, c = 'black')

ax.axvline(x = 3.5, color = 'black', linestyle = '--', linewidth = 1)

#ax.legend([bplot["boxes"][0], splot], ['distorted', 'empirical'], loc='upper left')

#ax.set_title('20%')
plt.show()
#fig.savefig("box_20%.png")

## 4. Tables in Section 6